In [1]:
#import necessary libraries
import requests
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import pprint
import numpy as np
import pandas as pd

In [ ]:
url1 = 'https://tabs.ultimate-guitar.com/tab/misc-soundtrack/a-star-is-born-shallow-chords-2488086'
url2 = 'https://tabs.ultimate-guitar.com/tab/jeff-buckley/hallelujah-chords-198052'
asession = AsyncHTMLSession() #creates Async object

r = await asession.get(url2,timeout=20) #requests website from server and waits for response
#r.html.render(timeout=20)
site = await r.html.arender(timeout=20) #renders in case there is javascript

soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
body = soup.find(class_='js-page js-global-wrapper') #finds main body of website which contains the strum patterns
patterns = body.find_all(class_='_2u0o0') #every strum pattern is within a class called '_2u0o0'


In [ ]:
patterns[0].contents

By cross checking with the website, I've determined that the classes that have '_1yupz' are downstrums, the classes that have '_3N-at' are downstrums, and the classes that have neither of those are beats you don't strum on

In [ ]:
print(len(patterns)) #the song I'm testing with has 3 strum patterns so this list should hold three Tags
print(type(patterns)) #confirming that patterns is a list/ResultSet
print(type(patterns[0].children)) #checking the type for the children of the Tag
print(type(next(patterns[0].children))) 
print(type(next(patterns[0].children)['class']))
print(next(patterns[0].children)['class'])#Returns class string as a list


In [ ]:
strumPatterns = [] #will hold all strum patterns for song with 1=downstrum, 2=upstrum, 0=skip

for pattern in patterns: #iterate through all strum patterns scraped
    currentPattern = [] #will hold current pattern
    count = 0
    for child in pattern.children: #iterate through all strums within the pattern
        className = child['class'] #get class name of the strum to determine if its upstrum, downstrum, or neither
        
        print(className) #print class name for debugging
        
        if '_1yupz' in className: #if it contains this string, it is an downstrum so append a 1
            currentPattern.append(1)
        elif '_3N-at' in className: #if it contains this string, it is an upstrum so append a 2
            currentPattern.append(2)
        else:   #if it contains this string, it is neither so append a 0
            currentPattern.append(0)
        
    strumPatterns.append(currentPattern) #add current pattern to list of all patterns in song
    print("Integer Pattern:",currentPattern)
    print()
           

In [ ]:
print(np.array(strumPatterns)) #should give 3x12 array because the song being tested has three strum patterns with 12 strums each

In [ ]:
#Now ill figure out how to scrape the links for all the top songs
#url for top daily chords songs:
#'https://www.ultimate-guitar.com/explore?type[]=Chords'
asession = AsyncHTMLSession() #creates Async object
URL = 'https://www.ultimate-guitar.com/explore?type[]=Chords'
r = await asession.get(URL,timeout=20) #requests website from server and waits for response

site = await r.html.arender(timeout=20) #renders in case there is javascript

soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html


In [ ]:
songs = soup.find_all(class_='_2KJtL _1mes3 kWOod') #'_1HdUE' is the class that holds the url for each song
print('Number of song links found:', len(songs)) #there should be 50 songs on the page

In [ ]:
print(type(songs[0]['href']),'\n') 
print(songs[0],'\n') #checks that songs[0] is a Tag
print(songs[0]['href']) #checks if href will return the url as string
print('\n',songs[0].text) #checks song name

In [2]:
#combine previous testing to make function that finds and returns strum patterns from given song url
import time



async def getPatterns(url): #function to parse for strum patterns at given url
    start_time = time.time()
    asession = AsyncHTMLSession() #creates Async object
    
    print("Requesting HTML")
    r = await asession.get(url,timeout=30) #requests website from server and waits for response
    #r.html.render(timeout=20)
    
    #print("Get Website --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    print("Rendering JavaScript")
    site = await r.html.arender(timeout=30) #renders the javascript, NOTE: Currently the speed bottleneck as the javascript is taking too long to render
    #site = r
    print("Render Website --- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
    body = soup.find(class_='js-page js-global-wrapper') #finds main body of website which contains the strum patterns
    patterns = body.find_all(class_='_2u0o0') #every strum pattern is within a class called '_2u0o0'
    
    #print("Parse With Soup --- %s seconds ---" % (time.time() - start_time))
    
    strumPatterns = [] #will hold all strum patterns for song with 1=downstrum, 2=upstrum, 0=skip
    
    start_time = time.time()
    
    for pattern in patterns: #iterate through all strum patterns scraped
        currentPattern = [] #will hold current pattern
        count = 0
        for child in pattern.children: #iterate through all strums within the pattern
            className = child['class'] #get class name of the strum to determine if its upstrum, downstrum, or neither
            if '_1yupz' in className: #if it contains this string, it is an downstrum so append a 1
                currentPattern.append(1)
            elif '_3N-at' in className: #if it contains this string, it is an upstrum so append a 2
                currentPattern.append(2)
            else:   #if it contains this string, it is neither so append a 0
                currentPattern.append(0)

        strumPatterns.append(currentPattern) #add current pattern to list of all patterns in song
    
    #print("Parse Strum Patterns --- %s seconds ---" % (time.time() - start_time)) 
    await r.session.close()
    await asession.close() #CHANGE MADE
    return strumPatterns #returns array of strum patterns, with each row being a strum pattern


In [ ]:
#test function
test = await getPatterns('https://tabs.ultimate-guitar.com/tab/green-day/boulevard-of-broken-dreams-chords-146744')
#test = await getPatterns('https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589')
test

In [ ]:
for song in songs:
    print(await getPatterns(song['href']),'\n')

In [ ]:
#input strum pattern as list with 0=skip,1=downstrum,2=upstrum, numSongs is the number of the top songs you want to check
async def searchStrumPattern(pattern,numSongs): 
    asession = AsyncHTMLSession() #creates Async object
    page = 1 #starts on page 1 of songs website
    URL = 'https://www.ultimate-guitar.com/explore?type[]=Chords&page='
    count = 0 #will hold # of songs checked
    while count < numSongs and page < 10: #each page holds 50 songs so sets limits on number of pages checked
        r = await asession.get(URL+str(page),timeout=30) #requests website from server and waits for response

        site = await r.html.arender(timeout=30) #renders in case there is javascript

        soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
        songs = soup.find_all(class_='_2KJtL _1mes3 kWOod') #'_1HdUE' is the class that holds the url for each song
        
        for song in songs:
            patterns = await getPatterns(song['href'])
            for row in patterns:
                if pattern == row:
                    print(song['href'],'\n',song.text,'\n')
            count += 1
            print(count)
        page += 1


In [ ]:
await searchStrumPattern([1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2],200) #should be strum pattern of Boulevard of Broken Dreams which is on
#[1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2]



In [ ]:
async def scrapePatterns(numSongs): 
    asession = AsyncHTMLSession() #creates Async object
    page = 1 #starts on page 1 of songs website
    URL = 'https://www.ultimate-guitar.com/explore?type[]=Chords&page='
    count = 0 #will hold # of songs checked
    data = []
    while count < numSongs and page < 10: #each page holds 50 songs so sets limits on number of pages checked
        r = await asession.get(URL+str(page),timeout=30) #requests website from server and waits for response
        
        site = await r.html.arender(timeout=30) #renders in case there is javascript

        soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
        songs = soup.find_all(class_='_2KJtL _1mes3 kWOod') #'_1HdUE' is the class that holds the url for each song
        print("check")
        for song in songs:
            try:
                print(song['href'])
                patterns = await getPatterns(song['href'])
                data.append((song.text,song['href'],patterns))
            except:
                print('Rendering Timeout after',count)
                
            count += 1
            if count > numSongs:
                break
            print(count,'\n')
        page += 1
    return data

In [ ]:
x = await scrapePatterns(400)

In [ ]:
x

In [3]:
async def getSongs(data,songs):
    for song in songs:
        try:
            print(song['href'])
            patterns = await getPatterns(song['href'])
            data.append((song.text,song['href'],patterns))
        except:
            print('Rendering Timeout')
                
    return data

In [4]:
async def scrapePatterns2(numpages): 
    asession = AsyncHTMLSession() #creates Async object
    page = 1 #starts on page 1 of songs website
    URL = 'https://www.ultimate-guitar.com/explore?type[]=Chords&page='
    data = []
    while page < numpages: #each page holds 50 songs so sets limits on number of pages checked
        print("\nPage:",page,'\n')
        r = await asession.get(URL+str(page),timeout=30) #requests website from server and waits for response
        
        site = await r.html.arender(timeout=30) #renders in case there is javascript

        soup = BeautifulSoup(r.html.html, features='lxml') #creates BeautifulSoup object which will parse the html
        songs = soup.find_all(class_='_2KJtL _1mes3 kWOod') #'_1HdUE' is the class that holds the url for each song
        print("check")
        data = await getSongs(data,songs)
        page += 1
        
    return data

In [ ]:
z = await scrapePatterns2(21)


Page: 1 

check
https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589
Requesting HTML
Rendering JavaScript
Render Website --- 2.709596872329712 seconds ---
https://tabs.ultimate-guitar.com/tab/elvis-presley/cant-help-falling-in-love-chords-1086983
Requesting HTML
Rendering JavaScript
Render Website --- 2.4569311141967773 seconds ---
https://tabs.ultimate-guitar.com/tab/jeff-buckley/hallelujah-chords-198052
Requesting HTML
Rendering JavaScript
Render Website --- 2.584188938140869 seconds ---
https://tabs.ultimate-guitar.com/tab/sia/snowman-chords-2233623
Requesting HTML
Rendering JavaScript
Render Website --- 3.0701382160186768 seconds ---
https://tabs.ultimate-guitar.com/tab/passenger/let-her-go-chords-1137467
Requesting HTML
Rendering JavaScript
Render Website --- 2.8498990535736084 seconds ---
https://tabs.ultimate-guitar.com/tab/eagles/hotel-california-chords-46190
Requesting HTML
Rendering JavaScript
Render Website --- 2.624398946762085 seconds ---
https://tabs.ul

check
https://tabs.ultimate-guitar.com/tab/lynyrd-skynyrd/sweet-home-alabama-chords-849467
Requesting HTML
Rendering JavaScript
Render Website --- 2.8843071460723877 seconds ---
https://tabs.ultimate-guitar.com/tab/creedence-clearwater-revival/have-you-ever-seen-the-rain-chords-56479
Requesting HTML
Rendering JavaScript
Render Website --- 7.193685054779053 seconds ---
https://tabs.ultimate-guitar.com/tab/coldplay/viva-la-vida-chords-675427
Requesting HTML
Rendering JavaScript
Render Website --- 2.896595001220703 seconds ---
https://tabs.ultimate-guitar.com/tab/ed-sheeran/i-see-fire-chords-1430599
Requesting HTML
Rendering JavaScript
Render Website --- 4.100667715072632 seconds ---
https://tabs.ultimate-guitar.com/tab/4-non-blondes/whats-up-chords-349210
Requesting HTML
Rendering JavaScript
Render Website --- 3.1909990310668945 seconds ---
https://tabs.ultimate-guitar.com/tab/misc-soundtrack/a-star-is-born-shallow-chords-2480119
Requesting HTML
Rendering JavaScript
Render Website --- 8.

Rendering JavaScript
Render Website --- 4.178552150726318 seconds ---

Page: 3 

check
https://tabs.ultimate-guitar.com/tab/don-mclean/american-pie-chords-187946
Requesting HTML
Rendering JavaScript
Render Website --- 3.938302993774414 seconds ---
https://tabs.ultimate-guitar.com/tab/calum-scott/you-are-the-reason-chords-2267805
Requesting HTML
Rendering JavaScript
Render Website --- 2.9081850051879883 seconds ---
https://tabs.ultimate-guitar.com/tab/clairo/softly-chords-2782903
Requesting HTML
Rendering JavaScript
Render Website --- 2.472568988800049 seconds ---
https://tabs.ultimate-guitar.com/tab/darius-rucker/wagon-wheel-chords-1215756
Requesting HTML
Rendering JavaScript
Render Website --- 3.2858731746673584 seconds ---
https://tabs.ultimate-guitar.com/tab/justin-bieber/anyone-chords-3494039
Requesting HTML
Rendering JavaScript
Render Website --- 6.801671028137207 seconds ---
https://tabs.ultimate-guitar.com/tab/r-e-m-/losing-my-religion-chords-114345
Requesting HTML
Rendering Jav

Render Website --- 3.8266842365264893 seconds ---

Page: 4 

check
https://tabs.ultimate-guitar.com/tab/oasis/wonderwall-chords-6125
Requesting HTML
Rendering JavaScript
Render Website --- 7.141776084899902 seconds ---
https://tabs.ultimate-guitar.com/tab/fugees/killing-me-softly-chords-50470
Requesting HTML
Rendering JavaScript
Render Website --- 3.312059164047241 seconds ---
https://tabs.ultimate-guitar.com/tab/the-beatles/here-comes-the-sun-chords-1726934
Requesting HTML
Rendering JavaScript
Render Website --- 3.692807912826538 seconds ---
https://tabs.ultimate-guitar.com/tab/sza/good-days-chords-3338225
Requesting HTML
Rendering JavaScript
Render Website --- 2.9971771240234375 seconds ---
https://tabs.ultimate-guitar.com/tab/radiohead/karma-police-chords-103398
Requesting HTML
Rendering JavaScript
Render Website --- 3.2399230003356934 seconds ---
https://tabs.ultimate-guitar.com/tab/bob-dylan/dont-think-twice-its-all-right-chords-178850
Requesting HTML
Rendering JavaScript
Render W

Render Website --- 3.3570518493652344 seconds ---

Page: 5 

check
https://tabs.ultimate-guitar.com/tab/wilbur-soot/im-in-love-with-an-e-girl-chords-2974481
Requesting HTML
Rendering JavaScript
Render Website --- 2.746422052383423 seconds ---
https://tabs.ultimate-guitar.com/tab/a-ha/take-on-me-chords-1842621
Requesting HTML
Rendering JavaScript
Render Website --- 3.544015884399414 seconds ---
https://tabs.ultimate-guitar.com/tab/coldplay/fix-you-chords-202594
Requesting HTML
Rendering JavaScript
Render Website --- 3.457143783569336 seconds ---
https://tabs.ultimate-guitar.com/tab/arctic-monkeys/whyd-you-only-call-me-when-youre-high-chords-1419146
Requesting HTML
Rendering JavaScript
Render Website --- 6.3907630443573 seconds ---
https://tabs.ultimate-guitar.com/tab/ed-sheeran/supermarket-flowers-chords-1956603
Requesting HTML
Rendering JavaScript
Render Website --- 3.9178290367126465 seconds ---
https://tabs.ultimate-guitar.com/tab/joseph-vincent/cant-take-my-eyes-off-you-chords-18835

Rendering JavaScript
Render Website --- 2.6009531021118164 seconds ---

Page: 6 

check
https://tabs.ultimate-guitar.com/tab/linkin-park/numb-chords-90873
Requesting HTML
Rendering JavaScript
Render Website --- 3.2487571239471436 seconds ---
https://tabs.ultimate-guitar.com/tab/kansas/dust-in-the-wind-chords-874870
Requesting HTML
Rendering JavaScript
Render Website --- 2.4998598098754883 seconds ---
https://tabs.ultimate-guitar.com/tab/audrey-hepburn/moon-river-chords-1788365
Requesting HTML
Rendering JavaScript
Render Website --- 2.753852128982544 seconds ---
https://tabs.ultimate-guitar.com/tab/niall-horan/this-town-chords-1879128
Requesting HTML
Rendering JavaScript
Render Website --- 2.76832914352417 seconds ---
https://tabs.ultimate-guitar.com/tab/billie-eilish/i-love-you-chords-2647653
Requesting HTML
Rendering JavaScript
Render Website --- 6.685976028442383 seconds ---
https://tabs.ultimate-guitar.com/tab/johnny-cash/i-walk-the-line-chords-278859
Requesting HTML
Rendering JavaS

Render Website --- 3.603872060775757 seconds ---

Page: 7 

check
https://tabs.ultimate-guitar.com/tab/britney-spears/toxic-chords-618385
Requesting HTML
Rendering JavaScript
Render Website --- 9.838854789733887 seconds ---
https://tabs.ultimate-guitar.com/tab/lukas-graham/7-years-chords-1752864
Requesting HTML
Rendering JavaScript
Render Website --- 4.396717071533203 seconds ---
https://tabs.ultimate-guitar.com/tab/queen/bohemian-rhapsody-chords-40606
Requesting HTML
Rendering JavaScript
Render Website --- 10.371336936950684 seconds ---
https://tabs.ultimate-guitar.com/tab/lp/lost-on-you-chords-1825835
Requesting HTML
Rendering JavaScript
Render Website --- 3.8342349529266357 seconds ---
https://tabs.ultimate-guitar.com/tab/the-beatles/a-day-in-the-life-chords-260796
Requesting HTML
Rendering JavaScript
Render Website --- 7.834846019744873 seconds ---
https://tabs.ultimate-guitar.com/tab/acdc/highway-to-hell-chords-139966
Requesting HTML
Rendering JavaScript
Render Website --- 4.83038

Rendering JavaScript
Render Website --- 9.634454011917114 seconds ---

Page: 8 

check
https://tabs.ultimate-guitar.com/tab/sixpence-none-the-richer/kiss-me-chords-2707
Requesting HTML
Rendering JavaScript
Render Website --- 4.329148769378662 seconds ---
https://tabs.ultimate-guitar.com/tab/taylor-swift/tis-the-damn-season-chords-3462035
Requesting HTML
Rendering JavaScript
Render Website --- 8.278136014938354 seconds ---
https://tabs.ultimate-guitar.com/tab/the-beatles/norwegian-wood-chords-1050930
Requesting HTML
Rendering JavaScript
Render Website --- 4.472692012786865 seconds ---
https://tabs.ultimate-guitar.com/tab/milky-chance/stolen-dance-chords-1243327
Requesting HTML
Rendering JavaScript
Render Website --- 4.785128831863403 seconds ---
https://tabs.ultimate-guitar.com/tab/r-e-m-/everybody-hurts-chords-37519
Requesting HTML
Rendering JavaScript
Render Website --- 3.6951651573181152 seconds ---
https://tabs.ultimate-guitar.com/tab/queen/dont-stop-me-now-chords-519549
Requesting 

Rendering JavaScript
Render Website --- 4.249167203903198 seconds ---

Page: 9 

check
https://tabs.ultimate-guitar.com/tab/poison/every-rose-has-its-thorn-chords-130272
Requesting HTML
Rendering JavaScript
Render Website --- 4.604298114776611 seconds ---
https://tabs.ultimate-guitar.com/tab/post-malone/stay-chords-2362315
Requesting HTML
Rendering JavaScript
Render Website --- 2.8937621116638184 seconds ---
https://tabs.ultimate-guitar.com/tab/elevation-worship/the-blessing-chords-3014201
Requesting HTML
Rendering JavaScript
Render Website --- 3.3847668170928955 seconds ---
https://tabs.ultimate-guitar.com/tab/daniela-andrade/la-vie-en-rose-chords-1722038
Requesting HTML
Rendering JavaScript
Render Website --- 3.705157995223999 seconds ---
https://tabs.ultimate-guitar.com/tab/stevie-wonder/isnt-she-lovely-chords-876425
Requesting HTML
Rendering JavaScript
Render Website --- 3.105756998062134 seconds ---
https://tabs.ultimate-guitar.com/tab/imagine-dragons/demons-chords-1149539
Request

In [8]:
dataframe = pd.DataFrame(z,columns=['Song','URL','StrumPattern'])
dataframe

,Song,URL,StrumPattern
0,Perfect,https://tabs.ultimate-guitar.com/tab/ed-sheera...,"[[1, 1, 1, 1, 1, 1], [1, 0, 0, 1, 0, 0, 1, 0, ..."
1,Cant Help Falling In Love,https://tabs.ultimate-guitar.com/tab/elvis-pre...,"[[1, 1, 1, 1, 1, 1]]"
2,Hallelujah (ver 2),https://tabs.ultimate-guitar.com/tab/jeff-buck...,"[[1, 0, 1, 2, 1, 2, 1, 0, 1, 2, 1, 2], [1, 0, ..."
3,Snowman (ver 3),https://tabs.ultimate-guitar.com/tab/sia/snowm...,"[[1, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1, 1]]"
4,Let Her Go,https://tabs.ultimate-guitar.com/tab/passenger...,"[[1, 0, 0, 0, 1, 2, 1, 2, 1, 0, 0, 0, 1, 2, 1,..."
...,...,...,...
395,A Thousand Years (ver 7),https://tabs.ultimate-guitar.com/tab/christina...,"[[1, 0, 1, 2, 1, 2]]"
396,Impossible,https://tabs.ultimate-guitar.com/tab/james-art...,"[[1, 0, 1, 2, 1, 0, 0, 2, 0, 2, 1, 0, 1, 0, 1,..."
397,Saline Solution,https://tabs.ultimate-guitar.com/tab/wilbur-so...,"[[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,..."
398,Losing Face (ver 2),https://tabs.ultimate-guitar.com/tab/wilbur-so...,"[[1, 2, 1, 2, 0, 2, 1, 2]]"


In [14]:
query1 = [1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2]
query2 = [1,0,1,2,0,2,1,2]

for num, entries in dataframe.iterrows():
    #print(entries[1],'\n')
    for pattern in entries[2]:
        if pattern == query2:
            print(entries[0],entries[1],'\n')

Your New Boyfriend https://tabs.ultimate-guitar.com/tab/wilbur-soot/your-new-boyfriend-chords-3297218 

Over The Rainbow (ver 3) https://tabs.ultimate-guitar.com/tab/israel-kamakawiwoole/over-the-rainbow-chords-2135261 

Sweet Child O Mine https://tabs.ultimate-guitar.com/tab/guns-n-roses/sweet-child-o-mine-chords-176076 

Radioactive https://tabs.ultimate-guitar.com/tab/imagine-dragons/radioactive-chords-1171909 

Falling https://tabs.ultimate-guitar.com/tab/harry-styles/falling-chords-2926901 

Wicked Game https://tabs.ultimate-guitar.com/tab/chris-isaak/wicked-game-chords-11066 

I Will Follow You Into The Dark (ver 3) https://tabs.ultimate-guitar.com/tab/death-cab-for-cutie/i-will-follow-you-into-the-dark-chords-335735 

Night Changes https://tabs.ultimate-guitar.com/tab/one-direction/night-changes-chords-1678440 

Redemption Song (ver 2) https://tabs.ultimate-guitar.com/tab/bob-marley/redemption-song-chords-417640 

Brown Eyed Girl (ver 4) https://tabs.ultimate-guitar.com/tab/van-

In [11]:
import pickle
pickle.dump(dataframe, open( "ScrapedSongs.p", "wb" ) )

In [27]:
def matchPatterns(dataframe, query):
    matches = []
    for num, entries in dataframe.iterrows():
    #print(entries[1],'\n')
        for pattern in entries[2]:
            if pattern == query2:
                matches.append((entries[0],entries[1]))
    result = pd.DataFrame(matches,columns=['Song','URL'])
    return result

In [28]:
query1 = [1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2]
query2 = [1,0,1,2,0,2,1,2]
test = pickle.load( open( "ScrapedSongs.p", "rb" ) )
matches = matchPatterns(test,query2)
matches

,Song,URL
0,Your New Boyfriend,https://tabs.ultimate-guitar.com/tab/wilbur-so...
1,Over The Rainbow (ver 3),https://tabs.ultimate-guitar.com/tab/israel-ka...
2,Sweet Child O Mine,https://tabs.ultimate-guitar.com/tab/guns-n-ro...
3,Radioactive,https://tabs.ultimate-guitar.com/tab/imagine-d...
4,Falling,https://tabs.ultimate-guitar.com/tab/harry-sty...
5,Wicked Game,https://tabs.ultimate-guitar.com/tab/chris-isa...
6,I Will Follow You Into The Dark (ver 3),https://tabs.ultimate-guitar.com/tab/death-cab...
7,Night Changes,https://tabs.ultimate-guitar.com/tab/one-direc...
8,Redemption Song (ver 2),https://tabs.ultimate-guitar.com/tab/bob-marle...
9,Brown Eyed Girl (ver 4),https://tabs.ultimate-guitar.com/tab/van-morri...
